In [1]:

"""
We use following lines because we are running on Google Colab
If you are running notebook on a local computer, you don't need this cell
"""
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow1/spoken_language_understanding/atis/main')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import tensorflow as tf
import pprint
import logging
import time
import numpy as np

from sklearn.metrics import classification_report, f1_score
from pathlib import Path

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

TensorFlow Version 1.14.0
GPU Enabled: True


In [0]:
def get_vocab(vocab_path):
  word2idx = {}
  with open(vocab_path) as f:
    for i, line in enumerate(f):
      line = line.rstrip()
      word2idx[line] = i
  return word2idx

In [0]:
def data_generator(f_path, params):
  print('Reading', f_path)
  with open(f_path) as f:
    for line in f:
      line = line.rstrip()
      text, slot_intent = line.split('\t')
      words = text.split()[1:-1]
      slot_intent = slot_intent.split()
      slots, intent = slot_intent[1:-1], slot_intent[-1]
      assert len(words) == len(slots)
      
      yield (words, (intent, slots))

In [0]:
def dataset(is_training, params):
  _shapes = ([None], ((), [None]))
  _types = (tf.string, (tf.string, tf.string))
  _pads = ('<pad>', ('_', 'O'))
  
  if is_training:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['train_path'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.shuffle(params['num_samples'])
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  else:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['test_path'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  
  return ds

In [0]:
def model_fn(features, labels, mode, params):
  is_training = (mode == tf.estimator.ModeKeys.TRAIN)
  
  vocab = tf.contrib.lookup.index_table_from_file(
    params['word_path'], num_oov_buckets=1)
  words = vocab.lookup(features)
  seq_len = tf.count_nonzero(words, 1, dtype=tf.int32)
  
  embedding = np.load(params['vocab_path'])
  embedding = tf.Variable(embedding, name='embedding', dtype=tf.float32)
  x = tf.nn.embedding_lookup(embedding, words)
  x = tf.layers.dropout(x, params['dropout_rate'], training=is_training)
  
  cell_fw = tf.nn.rnn_cell.GRUCell(params['rnn_units'])
  cell_bw = tf.nn.rnn_cell.GRUCell(params['rnn_units'])
  o, _ = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, x, seq_len, dtype=tf.float32)
  x = tf.concat(o, -1)
  
  y_intent = tf.layers.dense(tf.reduce_max(x, 1), params['intent_size'])
  y_slots = tf.layers.dense(x, params['slot_size'])

  crf_params = tf.get_variable(
    "transitions", [params['slot_size'], params['slot_size']], dtype=tf.float32)

  pred_ids, _ = tf.contrib.crf.crf_decode(y_slots, crf_params, seq_len)
  
  
  if labels is not None:
    intent, slots = labels
    vocab = tf.contrib.lookup.index_table_from_file(
      params['intent_path'], num_oov_buckets=1)
    intent = vocab.lookup(intent)
    vocab = tf.contrib.lookup.index_table_from_file(
      params['slot_path'], num_oov_buckets=1)
    slots = vocab.lookup(slots)
    
    loss_intent = tf.nn.sparse_softmax_cross_entropy_with_logits(
      labels=intent, logits=y_intent)
    loss_intent = tf.reduce_mean(loss_intent)
    
    log_likelihood, _ = tf.contrib.crf.crf_log_likelihood(
      y_slots, slots, seq_len, crf_params)
    loss_slots = tf.reduce_mean(-log_likelihood)
    
    loss_op = 10 * loss_intent + loss_slots

  
  if mode == tf.estimator.ModeKeys.TRAIN:
    variables = tf.trainable_variables()
    tf.logging.info('\n'+pprint.pformat(variables))
    
    grads = tf.gradients(loss_op, variables)
    grads, _ = tf.clip_by_global_norm(grads, params['clip_norm'])
    
    global_step=tf.train.get_or_create_global_step()
    decay_lr = tf.train.exponential_decay(
        params['lr'], global_step, 1000, .9)
    hook = tf.train.LoggingTensorHook({'lr': decay_lr}, every_n_iter=100)
    
    optim = tf.train.AdamOptimizer(decay_lr)
    train_op = optim.apply_gradients(
      zip(grads, variables), global_step=global_step)
    
    return tf.estimator.EstimatorSpec(mode=mode,
                                      loss=loss_op,
                                      train_op=train_op,
                                      training_hooks=[hook],)
  
  
  if mode == tf.estimator.ModeKeys.EVAL:
    return tf.estimator.EstimatorSpec(mode=mode,
                                      loss=loss_op)
  
  
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode,
      predictions={'intent': tf.argmax(y_intent, -1),
                   'slots': pred_ids})

In [0]:
params = {
  'model_dir': '../model/bigru_crf',
  'log_path': '../log/bigru_crf.txt',
  'train_path': '../data/atis.train.w-intent.iob',
  'test_path': '../data/atis.test.w-intent.iob',
  'word_path': '../vocab/word.txt',
  'vocab_path': '../vocab/word.npy',
  'intent_path': '../vocab/intent.txt',
  'slot_path': '../vocab/slot.txt',
  'batch_size': 16,
  'num_samples': 4978,
  'rnn_units': 300,
  'dropout_rate': 0.2,
  'clip_norm': 5.0,
  'lr': 3e-4,
  'num_patience': 3,
}

In [0]:
params['word2idx'] = get_vocab(params['word_path'])
params['intent2idx'] = get_vocab(params['intent_path'])
params['slot2idx'] = get_vocab(params['slot_path'])

params['word_size'] = len(params['word2idx']) + 1
params['intent_size'] = len(params['intent2idx']) + 1
params['slot_size'] = len(params['slot2idx']) + 1

In [0]:
def is_descending(history: list):
  history = history[-(params['num_patience']+1):]
  for i in range(1, len(history)):
    if history[i-1] <= history[i]:
      return False
  return True  

In [10]:
# Create directory if not exist
Path(os.path.dirname(params['log_path'])).mkdir(exist_ok=True)
Path(params['model_dir']).mkdir(exist_ok=True, parents=True)

# Logging
logger = logging.getLogger('tensorflow')
logger.setLevel(logging.INFO)
fh = logging.FileHandler(params['log_path'])
logger.addHandler(fh)

# Create an estimator
_eval_steps = params['num_samples']//params['batch_size'] + 1
config = tf.estimator.RunConfig(
  save_checkpoints_steps=_eval_steps,)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  model_dir=params['model_dir'],
  config=config,
  params=params)

# Train on training data and Evaluate on testing data
train_spec = tf.estimator.TrainSpec(
  input_fn=lambda: dataset(is_training=True, params=params),)

eval_spec = tf.estimator.EvalSpec(
  input_fn=lambda: dataset(is_training=False, params=params),
  steps=None,
  throttle_secs=10,)

best_f1 = .0
history_f1 = []
tf.enable_eager_execution()

while True:
  tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)
  
  intent = []
  slots = []
  for w, (i, s) in dataset(is_training=False, params=params):
    intent.append(i.numpy())
    slots.append(s.numpy())
  intent = [i for batch in intent for i in batch]
  intent = [params['intent2idx'].get(str(t, 'utf-8'), len(params['intent2idx'])) for t in intent]
  slots = [j for batch in slots for i in batch for j in i]
  slots = [params['slot2idx'].get(str(s, 'utf-8'), len(params['slot2idx'])) for s in slots]

  predicted = list(estimator.predict(input_fn=lambda: dataset(is_training=False, params=params)))
  y_slots = [j for i in predicted for j in i['slots']]
  y_intent = [i['intent'] for i in predicted]
  
  logger.info('\n'+classification_report(y_true = intent,
                                         y_pred = y_intent,
                                         labels = list(params['intent2idx'].values()),
                                         target_names = list(params['intent2idx'].keys()),
                                         digits=3))

  logger.info('\n'+classification_report(y_true = slots,
                                         y_pred = y_slots,
                                         labels = list(params['slot2idx'].values()),
                                         target_names = list(params['slot2idx'].keys()),
                                         sample_weight = np.sign(slots),
                                         digits=3))
  
  f1_slots = f1_score(y_true = slots,
                      y_pred = y_slots,
                      labels = list(params['slot2idx'].values()),
                      sample_weight = np.sign(slots),
                      average='micro',)
  history_f1.append(f1_slots)

  if f1_slots > best_f1:
    best_f1 = f1_slots
  logger.info("Best Slot F1: {:.3f}".format(best_f1))
  
  if len(history_f1) > params['num_patience'] and is_descending(history_f1):
    logger.info("Testing Slot F1 not improved over {} epochs, Early Stop".format(params['num_patience']))
    break

I0716 00:47:41.587580 140076727973760 estimator.py:209] Using config: {'_model_dir': '../model/bigru_crf', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 312, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f65d01917b8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
I0716 00:47:41.594257 140076727973760 estimator_training.py:186] Not using Distribute Coordi

Reading ../data/atis.train.w-intent.iob


I0716 00:47:50.547693 140076727973760 basic_session_run_hooks.py:262] loss = 87.49991, step = 0
I0716 00:47:50.549657 140076727973760 basic_session_run_hooks.py:262] lr = 0.0003
I0716 00:48:02.059066 140076727973760 basic_session_run_hooks.py:692] global_step/sec: 8.68664
I0716 00:48:02.065382 140076727973760 basic_session_run_hooks.py:260] loss = 18.045454, step = 100 (11.518 sec)
I0716 00:48:02.068305 140076727973760 basic_session_run_hooks.py:260] lr = 0.00029685578 (11.519 sec)
I0716 00:48:12.927720 140076727973760 basic_session_run_hooks.py:692] global_step/sec: 9.20081
I0716 00:48:12.934930 140076727973760 basic_session_run_hooks.py:260] loss = 17.184025, step = 200 (10.869 sec)
I0716 00:48:12.938170 140076727973760 basic_session_run_hooks.py:260] lr = 0.00029374452 (10.870 sec)
I0716 00:48:23.584062 140076727973760 basic_session_run_hooks.py:692] global_step/sec: 9.38412
I0716 00:48:23.586608 140076727973760 basic_session_run_hooks.py:260] loss = 4.921689, step = 300 (10.652 sec

Reading ../data/atis.test.w-intent.iob


I0716 00:48:28.744487 140076727973760 evaluation.py:275] Finished evaluation at 2019-07-16-00:48:28
I0716 00:48:28.746257 140076727973760 estimator.py:2039] Saving dict for global step 312: global_step = 312, loss = 9.044307
I0716 00:48:28.938807 140076727973760 estimator.py:2099] Saving 'checkpoint_path' summary for global step 312: ../model/bigru_crf/model.ckpt-312
I0716 00:48:28.991008 140076727973760 estimator.py:368] Loss for final step: 14.758665.


...

I0716 01:16:15.792758 140076727973760 evaluation.py:275] Finished evaluation at 2019-07-16-01:16:15
I0716 01:16:15.794436 140076727973760 estimator.py:2039] Saving dict for global step 10608: global_step = 10608, loss = 4.477148
I0716 01:16:15.801826 140076727973760 estimator.py:2099] Saving 'checkpoint_path' summary for global step 10608: ../model/bigru_crf/model.ckpt-10608
I0716 01:16:15.866571 140076727973760 estimator.py:368] Loss for final step: 0.00082361675.


Reading ../data/atis.test.w-intent.iob


I0716 01:16:16.192821 140076727973760 estimator.py:1145] Calling model_fn.
I0716 01:16:16.835687 140076727973760 estimator.py:1147] Done calling model_fn.
I0716 01:16:16.954639 140076727973760 monitored_session.py:240] Graph was finalized.
I0716 01:16:16.970279 140076727973760 saver.py:1280] Restoring parameters from ../model/bigru_crf/model.ckpt-10608
I0716 01:16:17.070095 140076727973760 session_manager.py:500] Running local_init_op.
I0716 01:16:17.086987 140076727973760 session_manager.py:502] Done running local_init_op.


Reading ../data/atis.test.w-intent.iob


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
I0716 01:16:20.199586 140076727973760 interactiveshell.py:2882] 
                                          precision    recall  f1-score   support

                             atis_flight      0.978     0.992     0.985       632
                            atis_airfare      0.979     0.979     0.979        48
                     atis_ground_service      1.000     1.000     1.000        36
                            atis_airline      1.000     1.000     1.000        38
                       atis_abbreviation      0.